In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

option = webdriver.ChromeOptions()
option.binary_location = '/opt/brave.com/brave/brave'

s = Service("./chromedriver-linux64/chromedriver")

driver = webdriver.Chrome(service=s, options=option)

In [2]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
driver.current_url

'data:,'

In [18]:
job = 'data-scientist'
location = 'Morocco'

driver.get(f"https://www.linkedin.com/jobs/{job}-jobs?location={location}&f_TPR=r2592000")
time.sleep(2)

driver.set_window_size(1366, 768)

In [22]:
for i in range(15):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(2)
    if i > 10:
        scroll_button = driver.find_element(By.CLASS_NAME, 'infinite-scroller__show-more-button')
        scroll_button.click()
        time.sleep(3)


In [32]:
jobs = {
    'Title':[],
    'Description':[],
    'Company':[],
    'Location':[],
    'Actively_Hiring': [],
    'ListDate':[],
    'DaysAgo':[],
    'Link':[]
}

ul_element = driver.find_element(By.CLASS_NAME, "jobs-search__results-list")
jobs_list = ul_element.find_elements(By.TAG_NAME, "li")
l = len(jobs_list)
print("Founded jobs:", l)

for i in tqdm(range(l)):
  try:  
    element = jobs_list[i]
    element.click()
    time.sleep(1.3)

    title = element.find_element(By.CLASS_NAME,"base-search-card__title").text
    description = driver.find_element(By.CLASS_NAME,"show-more-less-html__markup").text
    link = element.find_element(By.TAG_NAME,"a").get_attribute('href')
    Company = element.find_element(By.CLASS_NAME,"base-search-card__subtitle").text
    Location = element.find_element(By.CLASS_NAME,"job-search-card__location").text
    try:
      ListDate = element.find_element(By.CLASS_NAME,"job-search-card__listdate").text
    except Exception:
      ListDate = ""
    try:
      Hiring = "Yes" if element.find_element(By.CLASS_NAME, "job-search-card__benefits").text else "No"
    except Exception:
      Hiring = "No"

    try:
      duration = ListDate.split(" ")[1]
      period = int(ListDate.split(" ")[0]) * (1 if duration in ['days','day'] else 7 if duration in ['week','weeks'] else 30 if duration in ['month','months'] else 0)
    except Exception:
      period = np.nan

    jobs['Title'].append(title)
    jobs['Description'].append(description)
    jobs['Link'].append(link)
    jobs['Company'].append(Company)
    jobs['Location'].append(Location)
    jobs['ListDate'].append(ListDate)
    jobs['Actively_Hiring'].append(Hiring)
    jobs['DaysAgo'].append(period)
    
  except Exception as e:
    print(f"An exception occurred: {e}")
    break

df = pd.DataFrame(jobs)

Founded jobs: 274


100%|██████████| 274/274 [12:51<00:00,  2.81s/it]


In [34]:
today = pd.to_datetime('today').strftime('%Y-%m-%d')
df_ = df.sort_values('DaysAgo')
df_['Latest'] = 1
df_['Scraped'] = today
df_.to_csv(f"jobs_{today}.csv")
df_.head(10)

,Title,Description,Company,Location,Actively_Hiring,ListDate,DaysAgo,Link,Latest,Scraped
87,Ingénieur Full Stack/Ingénieure Full Stack,Entreprise :\nHPS est un leader mondialement r...,HPS,"Casablanca-Settat, Morocco",Yes,1 day ago,1.0,https://ma.linkedin.com/jobs/view/ing%C3%A9nie...,1,2023-12-23
185,Développeur Senior full stack,- Analyser et développer les composants en uti...,Telco DEV,"Fez, Fès-Meknès, Morocco",Yes,1 day ago,1.0,https://ma.linkedin.com/jobs/view/d%C3%A9velop...,1,2023-12-23
154,Data Engineer,Who are we ?\n\nArtefact is a new generation o...,Artefact,"Casablanca, Casablanca-Settat, Morocco",Yes,1 day ago,1.0,https://ma.linkedin.com/jobs/view/data-enginee...,1,2023-12-23
191,Développeur Senior Full Stack (DevOps) JAVA/An...,Responsabilités Principales\n\nAnalyser et com...,Astek Morocco,Rabat Metropolitan Area,Yes,1 day ago,1.0,https://ma.linkedin.com/jobs/view/d%C3%A9velop...,1,2023-12-23
138,Développeur Windev F/H,,VISEO,Morocco,Yes,1 day ago,1.0,https://ma.linkedin.com/jobs/view/d%C3%A9velop...,1,2023-12-23
100,CASA - IT / Unix systems Engineer (F/H),As part of DTIT (Digital Transformation & Info...,STMicroelectronics,"Prefecture of Casablanca, Casablanca-Settat, M...",Yes,1 day ago,1.0,https://ma.linkedin.com/jobs/view/casa-it-unix...,1,2023-12-23
96,INGENIEUR CONCEPTION & DEVELOPPEMENT JAVA/JEE,"RIBATIS , Opérateur de la transformation digit...",RIBATIS,"Mechouar, Casablanca-Settat, Morocco",No,1 day ago,1.0,https://ma.linkedin.com/jobs/view/ingenieur-co...,1,2023-12-23
225,ST Bouskoura – Ingénieur IT / Infrastructure O...,Missions:\n\nPart of Digital Transformation & ...,STMicroelectronics,"Prefecture of Casablanca, Casablanca-Settat, M...",Yes,1 day ago,1.0,https://ma.linkedin.com/jobs/view/st-bouskoura...,1,2023-12-23
70,Software Specialist Engineer,Would you like to make a difference to the env...,Eaton,"Casablanca, Casablanca-Settat, Morocco",Yes,1 day ago,1.0,https://ma.linkedin.com/jobs/view/software-spe...,1,2023-12-23
246,Développeur WordPress,Propos De Nous :\nDar Digital où l'innovation ...,DAR DIGITAL,"Agadir, Souss-Massa, Morocco",No,1 day ago,1.0,https://ma.linkedin.com/jobs/view/d%C3%A9velop...,1,2023-12-23


In [35]:
%load_ext sql
import csv, sqlite3

con = sqlite3.connect("JobsData.db")
cur = con.cursor()

In [36]:
%sql sqlite:///JobsData.db

In [37]:
df_.to_sql("JOBS", con, index=False, method="multi")
%sql PRAGMA table_info("JOBS")

 * sqlite:///JobsData.db
Done.


cid,name,type,notnull,dflt_value,pk
0,Title,TEXT,0,None,0
1,Description,TEXT,0,None,0
2,Company,TEXT,0,None,0
3,Location,TEXT,0,None,0
4,Actively_Hiring,TEXT,0,None,0
5,ListDate,TEXT,0,None,0
6,DaysAgo,REAL,0,None,0
7,Link,TEXT,0,None,0
8,Latest,INTEGER,0,None,0
9,Scraped,TEXT,0,None,0


In [43]:
%sql select * from jobs limit 2

 * sqlite:///JobsData.db
Done.


Title,Description,Company,Location,Actively_Hiring,ListDate,DaysAgo,Link,Latest,Scraped
Ingénieur Full Stack/Ingénieure Full Stack,"Entreprise :HPS est un leader mondialement reconnu dans l’édition des solutions de paiement électronique dédiées aux institutions financières.Grâce à sa plateforme globale utilisée par plus de 350 émetteurs, acquéreurs et switches nationaux, nous traitons tous types de cartes (crédit, débit, prépayées, fidélité, entreprise, carburant ...) via tous types de canaux (GAB, TPE Internet et mobile).",HPS,"Casablanca-Settat, Morocco",Yes,1 day ago,1.0,https://ma.linkedin.com/jobs/view/ing%C3%A9nieur-full-stack-ing%C3%A9nieure-full-stack-at-hps-3789408483?refId=PZg2iExorw0FQMmmqLrxrg%3D%3D&trackingId=4rh3mvPWC9zsawueFf5m%2Bw%3D%3D&position=13&pageNum=3&trk=public_jobs_jserp-result_search-card,1,2023-12-23
Développeur Senior full stack,"- Analyser et développer les composants en utilisant les langages appropriés,- Participer à la structuration des bases de données,- Définir et développer les protocoles et les scenarios de tests (tests unitaires, test de non régression et tests de charge),",Telco DEV,"Fez, Fès-Meknès, Morocco",Yes,1 day ago,1.0,https://ma.linkedin.com/jobs/view/d%C3%A9veloppeur-senior-full-stack-at-telco-dev-3764211024?refId=BXgqCJirAP1xCnHqv%2Fjo1w%3D%3D&trackingId=QqoMPlkzEsbT27b1Z7sb8Q%3D%3D&position=12&pageNum=7&trk=public_jobs_jserp-result_search-card,1,2023-12-23


In [44]:
con.close()
driver.quit()